In [ ]:
from refactor.dpo.model import DPOTrainModule

tac_gen_first = DPOTrainModule.load(
      'experiments/runs/lean_dojo_dpo/dpo_beamsearch_2023_12_05/10_09_00/checkpoints/epoch=0-step=10000-Pass@1_val=0.46.ckpt', device='cuda', freeze=True
)


In [ ]:
# generate input to test

tokenizer = tac_gen_first.tokenizer


In [ ]:
state = 'E : Type u_3,\n' \
      '_inst_1 : normed_add_comm_group E,\n' \
      'f : ℝ → E,\n' \
      'a b c d : ℝ,\n' \
      'μ : measure ℝ,\n' \
      'h : [c, d] ⊆ [a, b],\n' \
      'hf : integrable_on f (Ι a b) μ,\n' \
      'hc : ¬c = d\n' \
      '⊢ integrable_on f (Ι c d) μ'

In [ ]:
tokenized_input = tokenizer(state, return_tensors="pt", max_length=2048, truncation=True)

In [ ]:
tactic = 'have : <a>measure_theory.integrable_on</a> f (Ι c d) μ := hf.mono_set <a>set.Ioo_subset_uIoc_self</a>'

In [ ]:
tokenized_target =  tokenizer(tactic, return_tensors="pt", max_length=2048, truncation=True)


In [ ]:
import torch
rewards = torch.zeros(tokenized_target.input_ids.shape[1] - 1)

In [ ]:
rewards[-1] = 1

In [ ]:
rewards = rewards.unsqueeze(0)

In [ ]:
rewards.shape

In [ ]:
input = {'tokens': tokenized_input.input_ids.cuda(), 'attn_mask': tokenized_input.attention_mask.cuda(),
         'target': tokenized_target.input_ids.cuda(), 'target_mask': tokenized_target.attention_mask.cuda(),
         'rewards': rewards.cuda()}

In [ ]:
from models.iql_model import PerTokenIQL



iql = PerTokenIQL(tac_gen_first.generator, tac_gen_first.tokenizer).cuda()

iql.get_loss(input)